## Prerequisites

In [ ]:
# 파이썬을 아나콘다로 설치하신 분들은,
# 아래의 코드를 실행하면 LightGBM을 설치할 수 있습니다.
!conda install -c -y conda-forge lightgbm

In [ ]:
# 파이썬을 아나콘다로 설치하지 않은 분들은(그리고 pip가 설치되어 있는 분들은),
# 아래의 코드를 실행하면 LightGBM을 설치할 수 있습니다.
!pip install lightgbm

In [ ]:
# 아래 코드를 실행해보세요.
# 만일 실행이 되지 않는다면 다음의 링크 https://goo.gl/w45GHy 의 내용을 바탕으로 LightGBM을 재설치 해주세요.
import lightgbm

In [154]:
random_state = 42
random_state

42

## Otto Group Product Classification Challenge

이번 과제는 세계 최대의 전자상거래 회사 중 하나인 [Otto Group](https://www.ottogroup.com/)에서 주최하는 [Otto Group Product Classification Challenge](https://www.kaggle.com/c/otto-group-product-classification-challenge/) 경진대회에 참석해보겠습니다.

Otto Group은 익명화(anonymization)된 상품 정보에 대한 데이터를 제공하는데, 경진대회 참석자는 이 데이터를 활용하여 주어진 상품 카테고리(target)를 예측해야 합니다. 상품 카테고리는 Class_1부터 Class_9까지 총 9개가 있습니다. 주어진 데이터를 Decision Tree, Random Forest, 그리고 Gradient Boosting Machine를 활용하여 예측해보도록 하겠습니다.




In [155]:
import pandas as pd

data = pd.read_csv("data/train.csv", index_col="id")

print(data.shape)
data.head()

(61878, 94)


,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
4,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
5,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


## Preprocessing

In [156]:
label_name = "target"
label_name

'target'

In [157]:
feature_names = data.columns.difference([label_name])

print(len(feature_names))
feature_names

93


Index(['feat_1', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14',
       'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_2',
       'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25',
       'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_3', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_4', 'feat_40', 'feat_41',
       'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47',
       'feat_48', 'feat_49', 'feat_5', 'feat_50', 'feat_51', 'feat_52',
       'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58',
       'feat_59', 'feat_6', 'feat_60', 'feat_61', 'feat_62', 'feat_63',
       'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69',
       'feat_7', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74',
       'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_8',
       'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84

In [158]:
X = data[feature_names]

print(X.shape)
X.head()

(61878, 93)


,feat_1,feat_10,feat_11,feat_12,feat_13,feat_14,feat_15,feat_16,feat_17,feat_18,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_9,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,0,0,0,0,0,2,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,1,0,0,1,1,0,...,0,1,2,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,4,0,...,1,0,0,0,0,0,1,0,0,0


In [159]:
y = data[label_name]

print(y.unique())

print(y.shape)
y.head()

['Class_1' 'Class_2' 'Class_3' 'Class_4' 'Class_5' 'Class_6' 'Class_7'
 'Class_8' 'Class_9']
(61878,)


id
1    Class_1
2    Class_1
3    Class_1
4    Class_1
5    Class_1
Name: target, dtype: object

### Benchmark

데이터를 읽어왔다면 이제 Decision Tree, Random Forest, 그리고 Gradient Boosting Machine으로 상품의 카테고리를 예측해보겠습니다. 먼저 Decision Tree부터 사용하겠습니다.

In [160]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)

model.fit(X, y)
y_predict = model.predict(X)

print(y_predict.shape)
y_predict[0:5]

(61878,)


array(['Class_1', 'Class_1', 'Class_1', 'Class_1', 'Class_1'], dtype=object)

In [161]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y, y_predict)

print(f"Score = {score:.6f}")

Score = 1.000000


Error가 0이 나왔습니다! 이게 무슨 일일까요? 결과적으로 말씀드리자면, 데이터가 1개가 들어갈 때까지 분할을 멈추지 않고 계속 가지를 치기 때문입니다.


이렇게 현재 보유한 데이터(training set)를 잘 맞춘다고 해도, 앞으로 생길 미래의 데이터가 들어왔을때도 정확히 맞추리라는 보장이 없죠. 이를 모델이 과적합(over-fitting)되었다고 말합니다.

Decision Tree는 max_depth와 같은 하이퍼 파라미터를 조절하지 않는다면, 현재 보유한 데이터에서는 100%의 정확도를 보일 가능성이 있습니다. 즉, Decision Tree는 over-fitting에 취약합니다.

![](https://i.imgur.com/Ztaa056.png)

이번에는 Decision Tree가 현재 보유한 데이터(=이미 Decision Tree가 학습한 데이터) 뿐만 아니라, 학습하지 않은 미래의 데이터도 잘 예측하는지 판단하기 위해 Hold-out Validation으로 모델을 검증해보겠습니다.

In [162]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(49502, 93) (49502,)
(12376, 93) (12376,)


In [163]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state=random_state)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
train_error = accuracy_score(y_train, y_train_predict)

y_test_predict = model.predict(X_test)
test_error = accuracy_score(y_test, y_test_predict)

print(f"Error(train) = {train_error:.6f}")
print(f"Error(test) = {test_error:.6f}")

Error(train) = 1.000000
Error(test) = 0.713478


train error는 변함없이 1.0이지만, test error는 1보다 낮은 숫자가 나오는 것을 확인할 수 있습니다.

---

인공지능 알고리즘의 학습 곡선은 일반적으로 다음과 같은 모양을 띕니다

![](https://i.imgur.com/AAYelnt.png)

모델이 복잡해질 수록 training data의 오류는 줄여나가지만, 점차 과적합이 되기 시작하면서 test set에 대한 오류는 증가하기 시작하죠. 우리는 나중에 보지못한 (Unseen) 데이터를 잘 맞추는 것을 목표로 하고 있기 때문에 test set에 대한 오류를 실제 모델의 성능으로 생각해야합니다.

이번에는 test error를 낮추기 위해서 Decision Tree의 최대 깊이(max_depth)를 제한해보겠습니다.

In [164]:
model = DecisionTreeClassifier(max_depth=20, random_state=random_state)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
train_error = accuracy_score(y_train, y_train_predict)

y_test_predict = model.predict(X_test)
test_error = accuracy_score(y_test, y_test_predict)

print("Use Decision Tree. max_depth=20")

print(f"Error(train) = {train_error:.6f}")
print(f"Error(test) = {test_error:.6f}")

Use Decision Tree. max_depth=20
Error(train) = 0.826593
Error(test) = 0.721396


max_depth를 제한했더니 train error는 줄어들었지만, test error는 오히려 증가했습니다. 그러므로 좋은 test  error가 나오는 적절한 max_depth 값을 조정해주는 것이 중요하다는 것을 알 수 있었습니다.

이번에는 Decision Tree가 아닌, Random Forest를 통해 예측해보겠습니다.

In [122]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=random_state)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
train_error = accuracy_score(y_train, y_train_predict)

y_test_predict = model.predict(X_test)
test_error = accuracy_score(y_test, y_test_predict)

print(f"Error(train) = {train_error:.6f}")
print(f"Error(test) = {test_error:.6f}")

Use Random Forest
Error(train) = 0.992445
Error(test) = 0.782401


아무런 hyperparameter의 조정을 거치지 않아도 Decision Tree보다 더 좋은 성능을 보인다는 것을 알 수 있습니다. 이는 Random Forest의 Bagging 알고리즘이 Decision Tree의 단점을 보완해줌으로서 모델을 더욱 견고하게 만들어주기 때문입니다.

마찬가지로 Gradient Boosting Machine의 구현체인 LightGBM으로 성능을 측정해보겠습니다.

In [171]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train)

y_train_predict = model.predict(X_train)
train_error = accuracy_score(y_train, y_train_predict)

y_test_predict = model.predict(X_test)
test_error = accuracy_score(y_test, y_test_predict)

print(f"Error(train) = {train_error:.6f}")
print(f"Error(test) = {test_error:.6f}")

Error(train) = 0.848329
Error(test) = 0.805753


벤치마킹 결과, LightGBM > Random Forest > Decision Tree 순으로 성능이 좋아지는 것을 확인할 수 있었습니다.